In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import skimage.io as io
from tqdm import tqdm
%matplotlib qt

In [116]:
from xrdmaptools.utilities.utilities import arbitrary_center_of_mass

fit_centers = np.asarray([[x0, y0] for x0, y0, _, _ in fit_params])
fit_radii = [R0 for _, _, R0, _ in fit_params]
fit_weights = [np.sum(np.array(blob_sizes)[blob_nums]) for _, _, _, blob_nums in fit_params]
mean_arc_center = arbitrary_center_of_mass(fit_weights, *fit_centers.T)
mean_arc_radius = arbitrary_center_of_mass(fit_weights, fit_radii)

In [117]:
import matplotlib
from matplotlib import cm

fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

im = ax.imshow(calib_img)
fig.colorbar(im, ax=ax)

norm = matplotlib.colors.Normalize(vmin=0, vmax=(len(fit_params)))
mapper = cm.scalerMappable(norm=norm, cmap='tab20')
circle_colors = [(r, g, b) for r, g, b, a in mapper.to_rgba(range(len(fit_params)))]

for i, param in enumerate(fit_params):

    if fit_weights[i] < np.max(fit_weights) * 0.25:
        continue

    x0, y0, R0, _ = param

    #circles.append(plt.Circle((x0, y0), R0, color='b', fill=True))
    ax.add_patch(plt.Circle((x0, y0), R0, color=circle_colors[i], fill=False))
    #ax.scatter(x0, y0, color=circle_colors[i], s=10)

fig.show()

In [ ]:
# Attempt to connect smaller isolated blobs
xc, yc = mean_arc_center

minor_blobs = []
dropped_fits = []
for i, weight in enumerate(fit_weights):
    
    if weight < np.max(fit_weights) * 0.25:
        _, _, _, blob_nums = fit_params[i]
        minor_blobs.extend(blob_nums)
        dropped_fits.append(i)

# Drop insignificant fits
for index in sorted(dropped_fits, reverse=True):
    del fit_params[index]
    del fit_radii[index]
    del fit_weights[index]
fit_centers = np.delete(fit_centers, dropped_fits, axis=0)

In [121]:
candidate_blobs = []
candidate_radii = []
for i, minor_blob in enumerate(minor_blobs):
    
    # Find approximate radius
    y, x = np.array(np.where(blob_image == minor_blob))
    Ri = np.sqrt((x - xc)**2 + (y - yc)**2)
    minor_radius = np.mean(Ri)

    # Check for first pass
    if len(candidate_radii) < 1:
        candidate_radii.append(minor_radius)
        candidate_blobs.append([minor_blob])
        continue
    
    FITS_OTHER_ARC = False
    for ii, R0 in enumerate(candidate_radii):
        if np.abs(minor_radius - R0) < 0.05 * R0:
            candidate_blobs[ii].append(minor_blobs[i])
            FITS_OTHER_ARC = True
            break
    
    if FITS_OTHER_ARC:
        continue
    else:
        candidate_radii.append(minor_radius)
        candidate_blobs.append([minor_blob])

candidate_weights = [np.sum(np.array(blob_sizes)[blob_nums]) for blob_nums in candidate_blobs]


candidate_fits = []
for i in range(len(candidate_radii)):
    # Add a cut-off to ignore arcs that are too insignificant
    # Blobs that are not well-aligned along an arc should not contribute significantly
    if candidate_weights[i] < np.mean(fit_weights) * 0.25:
        print('insignificant')
        print(candidate_weights[i])
        #continue


    # Combine minor blobs and fit arc
    combined_blobs = np.sum([blob_image == num for num in candidate_blobs[i]], axis=0)
    y, x = np.array(np.where(combined_blobs))
    x0, y0, R0 = leastsq_arc(x, y)

    # Check for reasonable center
    dist = np.sqrt((x0 - xc)**2 + (y0 - yc)**2)
    if dist > 0.85 * mean_arc_radius[0]:
        print('bad center')
        continue

    candidate_fits.append([x0, y0, R0, candidate_blobs[i]])

insignificant
673
insignificant
466
insignificant
318


In [122]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

im = ax.imshow(calib_img)
fig.colorbar(im, ax=ax)

for param in fit_params:
    x0, y0, R0, _ = param

    #circles.append(plt.Circle((x0, y0), R0, color='b', fill=True))
    ax.add_patch(plt.Circle((x0, y0), R0, color='r', fill=False))
    #ax.scatter(x0, y0, c='r', s=10)

for param in candidate_fits:
    x0, y0, R0, _ = param

    #circles.append(plt.Circle((x0, y0), R0, color='b', fill=True))
    ax.add_patch(plt.Circle((x0, y0), R0, color='g', fill=False))
    #ax.scatter(x0, y0, c='g', s=10)

fig.show()

In [126]:
arc_image = np.zeros_like(blob_image)

final_fits = fit_params + candidate_fits

arc_centers = np.asarray([[x0, y0] for x0, y0, _, _ in final_fits])
arc_radii = [R0 for _, _, R0, _ in final_fits]
arc_sizes = [np.sum(np.array(blob_sizes)[blob_nums]) for _, _, _, blob_nums in final_fits]
mean_arc_center = arbitrary_center_of_mass(arc_sizes, *arc_centers.T)

for i, (radius, fit) in enumerate(sorted(zip(arc_radii, final_fits))):
    x0, y0, R0, blob_nums = fit
    for num in blob_nums:
        arc_image[blob_image == num] = i + 1

In [128]:
plot_image(arc_image)

In [76]:
from xrdmaptools.reflections.spot_blob_search import spot_search
from xrdmaptools.utilities.image_corrections import rescale_array

spots = spot_search(rescale_array(calib_img.astype(np.float32), arr_min=0, upper=1),
                    expansion=0,
                    threshold_method='minimum',
                    size=0,
                    multiplier=2,
                    plotme=True)

In [77]:
x_lst, y_lst, ring_lst = [], [], []
for spot in spots[0]:
    if int(arc_image[*spot]) != 0:
        x_lst.append(spot[0])
        y_lst.append(spot[1])
        ring_lst.append(int(arc_image[*spot]))

data = np.array([x_lst, y_lst, ring_lst]).T

In [94]:
data[:, -1] += 1

In [95]:
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator
from pyFAI.calibrant import Calibrant
from pyFAI.geometryRefinement import GeometryRefinement

from xrdmaptools.geometry.area_detectors import Dexela2315
from xrdmaptools.utilities.math import energy_2_wavelength

wavelength = energy_2_wavelength(15) * 1e-10

reference_file = 'D:\\Musterman_postdoc\\zincite_extended.txt'
calibrant = Calibrant(filename=reference_file, wavelength=wavelength)
dexela2315 = Dexela2315()
dexela2315.set_binning(bin_size=(4, 4))

#tr = GeometryRefinement(data=data, detector=dexela2315, calibrant=calibrant, wavelength=energy_2_wavelength(18)*1e-10,
#                                                dist=0.425, poni1=0.07, poni2=0.16, rot1=0.54, rot2=0, rot3=0)
tr = GeometryRefinement(data=data, detector=dexela2315, calibrant=calibrant, wavelength=wavelength)

In [96]:
tr.refine3(fix='wavelength') # ALWAYS fix wavelength. Refinement yields distances and energy otherwise

0.00014448402210346905

In [97]:
fig, ax = plt.subplots(2, 1, figsize=(10, 5), dpi=200, sharex=True)

res = tr.integrate2D_ng(calib_img, 4096, 360, unit='2th_deg')
tth, chi = res[1], res[2]

im = ax[0].imshow(res[0], extent=[tth[0], tth[-1], chi[0], chi[-1]], vmin=0, aspect='auto')
fig.colorbar(im, ax=ax)
ax[0].set_xlabel('Scattering Angle, 2θ [°]')
ax[0].set_ylabel('Azimuthal Angle, χ [°]')

ax[1].plot(*tr.integrate1D_ng(calib_img, 4096, unit='2th_deg'))


fig.show()

In [98]:
plt.close('all')

In [850]:
arc_radii += np.sqrt(np.subtract(*((arc_centers - mean_arc_center)**2).T))

In [851]:
arc_radii

array([1013.86849607, 1322.83408748, 1157.60615967, 1250.47249592,
       1338.70456918, 1300.73879965, 2365.45003633, 1643.45497526,
       1433.27730033, 1809.60298106])

In [852]:
d_spacings = np.genfromtxt('D:\\Musterman_postdoc\\zincite_extended.txt')

In [853]:
from xrdmaptools.utilities.math import d_2_tth, energy_2_wavelength

arc_guess = np.tan(np.radians(d_2_tth(d_spacings, wavelength=energy_2_wavelength(12))))

In [869]:
d_2_tth(d_spacings, wavelength=energy_2_wavelength(12))

array([21.15698695, 22.89426123, 24.08571677, 31.36782168, 37.07293765,
       40.94571218, 43.08173403, 44.03524455, 44.71853186, 46.75342327,
       49.32679631, 51.87960735, 56.41223446, 58.09402374, 59.44215036,
       61.12332697, 63.26485142, 63.84374127, 65.41867221, 66.8365714 ,
       69.4406206 , 71.61331304, 73.04839778, 76.2246717 , 77.09473659,
       77.6498454 , 79.01926186])

In [862]:
from scipy.optimize import curve_fit

def vector_transform(vector, scale, shift):
    vector = np.asarray(vector)
    return vector * scale + shift

def vector_transform(vector, a, b, c):
    vector = np.asarray(vector)
    return a * vector * vector + b * vector + c

shift_list = []
mse_list = []
fit_list = []
for ring_shift in range(9):
    selected_rings = arc_guess[ring_shift:len(arc_radii) + ring_shift]

    popt, pcov = curve_fit(vector_transform, sorted(arc_radii), selected_rings)
    mse = np.mean((selected_rings - vector_transform(sorted(arc_radii), *popt))**2)
    mse_list.append(mse)
    shift_list.append(ring_shift)
    fit_list.append(popt)

In [863]:
mse_list

[0.009338979622339032,
 0.007068774121120808,
 0.0028056216530133226,
 0.0005341724063492039,
 0.0018578795615089283,
 0.005151222489066958,
 0.009313712804782201,
 0.011483903794765507,
 0.009964646363425542]

In [864]:
fig, ax = plt.subplots(1,1, figsize=(5, 5), dpi=200)

ax.vlines(arc_guess, 0, 1, label='calc', color='k')
ax.vlines(vector_transform(sorted(arc_radii), *fit_list[np.argmin(mse_list)]), 0, 1, label='fit', color='r')

ax.legend()

fig.show()

In [858]:
from scipy.optimize import minimize


def vector_transform(vector, scale, shift):
    vector = np.asarray(vector)
    return vector * scale + shift

def fit_vector_error(args):
    scale, shift = args
    fit_diff = (vector_transform(sorted(arc_radii), scale, shift)[np.newaxis, :]
                - arc_guess[:, np.newaxis])
    return np.min(fit_diff**2, axis=0).sum()

scale_guess = np.mean(arc_guess[:len(arc_radii)] / sorted(arc_radii))
scale_guess = 0
shift_guess = 0.31
shift_guess = 0

res = minimize(fit_vector_error, (scale_guess, shift_guess), bounds=((0, None), (None, None)), method='Powell')
res

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 0.02663957773175754
       x: [ 1.448e-03 -1.730e-01]
     nit: 3
   direc: [[ 1.000e+00  0.000e+00]
           [ 7.992e-05 -1.170e-01]]
    nfev: 110

In [865]:
def quadratic_vector_transform(vector, a, b, c):
    vector = np.asarray(vector)
    return a * vector * vector + b * vector + c

def fit_vector_error(args):
    a, b, c = args
    fit_diff = (quadratic_vector_transform(sorted(arc_radii), a, b, c)[np.newaxis, :]
                - arc_guess[:, np.newaxis])
    return np.min(fit_diff**2, axis=0).sum()

res = minimize(fit_vector_error, (0, 0, 0), bounds=((None, None), (0, None), (None, None)), method='Powell')
res

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 0.020309311791531965
       x: [ 5.937e-07  1.202e-03 -9.076e-01]
     nit: 2
   direc: [[ 1.000e+00  0.000e+00  0.000e+00]
           [ 0.000e+00  1.000e+00  0.000e+00]
           [ 0.000e+00  0.000e+00  1.000e+00]]
    nfev: 90

In [866]:
fig, ax = plt.subplots(1,1, figsize=(5, 5), dpi=200)

ax.vlines(arc_guess, 0, 1, label='calc', color='k')
ax.vlines(quadratic_vector_transform(sorted(arc_radii), *res['x']), 0, 1, label='fit', color='r')

ax.legend()

fig.show()

In [860]:
fig, ax = plt.subplots(1,1, figsize=(5, 5), dpi=200)

ax.vlines(arc_guess, 0, 1, label='calc', color='k')
ax.vlines(vector_transform(sorted(arc_radii), *res['x']), 0, 1, label='fit', color='r')

ax.legend()

fig.show()

In [861]:
from matplotlib.widgets import Slider

fig, ax = plt.subplots(1,1, figsize=(5, 5), dpi=200)

ax.vlines(tth_guess, 0, 1, label='calc', color='k')
lines = ax.vlines(vector_transform(sorted(arc_radii), *res['x']), 0, 1, label='fit', color='r')

ax.legend()

# Make a vertically oriented slider to control the amplitude
axscale = fig.add_axes([0.25, 0.05, 0.65, 0.03])
scale_slider = Slider(
    ax=axscale,
    label="Scale",
    valmin=0,
    valmax=1e-1,
    valinit=1e-2,
)

# Make a vertically oriented slider to control the amplitude
axshift = fig.add_axes([0.25, 0, 0.65, 0.03])
shift_slider = Slider(
    ax=axshift,
    label="Shift",
    valmin=-1e2,
    valmax=1e2,
    valinit=0
)

def update_segments(LineCollection):
    seg = np.asarray(LineCollection.get_segments())
    scale = scale_slider.val
    shift = shift_slider.val
    pos = vector_transform(sorted(arc_radii), scale, shift)
    seg[:, 0, 0] = pos
    seg[:, 1, 0] = pos
    return seg

def update(val):
    lines.set_segments(update_segments(lines))            
    fig.canvas.draw_idle()

# register the update function with each slider
scale_slider.on_changed(update)
shift_slider.on_changed(update)

fig.show()

In [640]:
plot_image(calc_ratios)

In [703]:
measrued_ratios = np.array(sorted(arc_radii))[:, np.newaxis] / np.array(sorted(arc_radii))[np.newaxis, :]
q_guess = np.array(sorted(arc_radii))[:, np.newaxis] / np.array(sorted(arc_radii))[np.newaxis, :]

In [653]:
from scipy.ndimage import correlate

corr = correlate(calc_ratios, measured_ratios, mode='constant', cval=-100)
plot_image(corr)

In [652]:
from scipy.ndimage import convolve

conv = convolve(calc_ratios, measured_ratios, mode='constant', cval=-100)
plot_image(conv)

In [633]:
vector_transform(sorted(arc_radii), *res['x'])

array([0.44017572, 0.49372161, 0.51908545, 0.53673662, 0.54685997,
       0.57916596, 0.62032753, 0.66464039, 0.74430497, 0.77949846,
       0.80383704, 0.83820329, 0.89976101, 0.90727005, 0.96593787,
       0.98646775, 1.05089315, 1.11289052, 1.21192884])

In [617]:
plot_image(calib_img)

In [526]:
x0, y0, R0 = algebraic_arc(pixels[1], pixels[0])
#x0, y0, R0 = leastsq_arc(pixels[1], pixels[0])
print(f'Center is found at ({x0:.2f}, {y0:.2f}) with radius {np.round(R0, 0)}.')

Center is found at (1402.76, 354.95) with radius 831.0.


In [525]:
#x0, y0, R0 = algebraic_arc(pixels[1], pixels[0])
x0, y0, R0 = leastsq_arc(pixels[1], pixels[0])
print(f'Center is found at ({x0:.2f}, {y0:.2f}) with radius {np.round(R0, 0)}.')

Center is found at (1445.19, 360.94) with radius 873.0.


In [8]:
def algebraic_arc(x, y):
    # coordinates of the barycenter
    x_m = np.mean(x)
    y_m = np.mean(y)

    # calculation of the reduced coordinates
    u = x - x_m
    v = y - y_m

    # linear system defining the center (uc, vc) in reduced coordinates:
    #    Suu * uc +  Suv * vc = (Suuu + Suvv)/2
    #    Suv * uc +  Svv * vc = (Suuv + Svvv)/2
    Suv  = np.sum(u*v)
    Suu  = np.sum(u**2)
    Svv  = np.sum(v**2)
    Suuv = np.sum(u**2 * v)
    Suvv = np.sum(u * v**2)
    Suuu = np.sum(u**3)
    Svvv = np.sum(v**3)

    # Solving the linear system
    A = np.array([ [ Suu, Suv ], [Suv, Svv]])
    B = np.array([ Suuu + Suvv, Svvv + Suuv ]) / 2.0
    uc, vc = np.linalg.solve(A, B)

    xc_1 = x_m + uc
    yc_1 = y_m + vc

    # Calcul des distances au centre (xc_1, yc_1)
    Ri_1     = np.sqrt((x - xc_1)**2 + (y - yc_1)**2)
    R_1      = np.mean(Ri_1)
    residu_1 = np.sum((Ri_1 - R_1)**2)

    return xc_1, yc_1, R_1

In [9]:
def leastsq_arc(x, y):
    from scipy import optimize

    def calc_R(xc, yc):
        """ calculate the distance of each 2D points from the center (xc, yc) """
        return np.sqrt((x - xc)**2 + (y - yc)**2)

    def f_2(c):
        """ calculate the algebraic distance between the data points and the mean circle centered at c=(xc, yc) """
        Ri = calc_R(*c)
        return Ri - np.mean(Ri)

    center_estimate = np.mean(x), np.mean(y)
    center_2, ier = optimize.leastsq(f_2, center_estimate)

    xc_2, yc_2 = center_2
    Ri_2 = calc_R(*center_2)
    R_2 = np.mean(Ri_2)
    residu_2 = sum((Ri_2 - R_2)**2)
    return xc_2, yc_2, R_2

In [523]:
xc_2

1445.1912784768522

In [326]:
pixels = np.array(np.where(minimum_filter(proc_img > 0.5, size=3)))
#pixels = peak_local_max(proc_img, threshold_abs=0.5).T

In [327]:
pixels.shape

(2, 10744)

In [328]:
from xrdmaptools.utilities.utilities import label_nearest_spots

out = label_nearest_spots(pixels.T, max_dist=5)
print(len(np.unique(out.T[2])))

282


In [329]:
fig, ax = plot_image(calib_img, vmin=0, vmax=50, return_plot=True)

ax.scatter(out.T[1], out.T[0], c=out.T[2], s=1, cmap='tab20')

fig.show()

In [547]:
fig, ax = plot_image(calib_img, vmin=0, vmax=50, return_plot=True)

ring_mask = out.T[2] == 25
ax.scatter(out.T[1][ring_mask], out.T[0][ring_mask], c=out.T[2][ring_mask], s=1, cmap='jet_r')

fig.show()

In [260]:
from xrdmaptools.reflections.spot_blob_search import spot_search
from xrdmaptools.utilities.image_corrections import rescale_array

spots = spot_search(rescale_array(calib_img.astype(np.float32), arr_min=0, upper=1), expansion=0, threshold_method='minimum', size=0, multiplier=2, plotme=True)

In [261]:
out = label_nearest_spots(spots[0], max_dist=10)
print(len(np.unique(out.T[2])))

351


In [259]:
fig, ax = plot_image(calib_img, vmin=0, vmax=50, return_plot=True)

ax.scatter(out.T[1], out.T[0], c=out.T[2], s=5, cmap='tab20')

fig.show()

In [189]:
out = label_nearest_spots(spots[0], max_dist=5)
print(len(np.unique(out.T[2])))

307


In [190]:
fig, ax = plot_image(calib_img, vmin=0, vmax=50, return_plot=True)

ax.scatter(out.T[1], out.T[0], c=out.T[2], s=5, cmap='tab20')

fig.show()

In [44]:
out

array([[  0.,  89.,   1.],
       [  0.,  90.,   1.],
       [  0.,  91.,   1.],
       ...,
       [485., 663.,  17.],
       [485., 664.,  17.],
       [485., 665.,  17.]])

In [33]:
reference_file = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\DOE_SCGSR\\Data\\221128 Beamtime\\maps\\zincite_extended.txt'''
calibrant = Calibrant(filename=reference_file, wavelength=wavelength)
dexela2315 = Dexela2315()
dexela2315.set_binning(bin_size=(4, 4))

#tr = GeometryRefinement(data=data, detector=dexela2315, calibrant=calibrant, wavelength=energy_2_wavelength(18)*1e-10,
#                                                dist=0.425, poni1=0.07, poni2=0.16, rot1=0.54, rot2=0, rot3=0)
tr = GeometryRefinement(data=data, detector=dexela2315, calibrant=calibrant, wavelength=wavelength)

In [34]:
print(tr.param, tr.energy)

[0.21713423606178558, 0.06912313131536871, 0.1719973269265199, 0.6891260297723616, 0.010974966747997314, 0.0] 14.999999999999998


In [35]:
tr.refine3(fix='wavelength') # ALWAYS fix wavelength. Refinement yields distances and energy otherwise

0.00044931509613461046

In [36]:
print(tr.param, tr.energy)

[0.36015035216298746, 0.07750011184365128, -0.0005245886019737451, 0.1799899634142498, -0.009077382334506362, -1.8221972238451893e-06] 14.999999999999998


In [37]:
res = tr.integrate2D_ng(fixed_img, 1024, 360, unit='2th_deg')
tth, chi = res[1], res[2]

In [38]:
fig, ax = plt.subplots(2, 1, figsize=(10, 5), dpi=200, sharex=True)

res = tr.integrate2D_ng(fixed_img, 4096, 360, unit='2th_deg')
tth, chi = res[1], res[2]

im = ax[0].imshow(res[0], extent=[tth[0], tth[-1], chi[0], chi[-1]], vmin=0, aspect='auto')
fig.colorbar(im, ax=ax)
ax[0].set_xlabel('Scattering Angle, 2θ [°]')
ax[0].set_ylabel('Azimuthal Angle, χ [°]')

ax[1].plot(*tr.integrate1D_ng(fixed_img, 4096, unit='2th_deg'))


fig.show()

In [54]:
import numpy as np
import matplotlib.pyplot as plt

def cartesian_ellipse(xy, a, b, c, d, e, f):
    x = xy[0]
    y = xy[1]

    return 


def fit_ellipse(x, y):
    """

    Fit the coefficients a,b,c,d,e,f, representing an ellipse described by
    the formula F(x,y) = ax^2 + bxy + cy^2 + dx + ey + f = 0 to the provided
    arrays of data points x=[x1, x2, ..., xn] and y=[y1, y2, ..., yn].

    Based on the algorithm of Halir and Flusser, "Numerically stable direct
    least squares fitting of ellipses'.


    """

    D1 = np.vstack([x**2, x * y, y**2]).T
    D2 = np.vstack([x, y, np.ones(len(x))]).T
    S1 = D1.T @ D1
    S2 = D1.T @ D2
    S3 = D2.T @ D2
    T = -np.linalg.inv(S3) @ S2.T
    M = S1 + S2 @ T
    C = np.array(((0, 0, 2), (0, -1, 0), (2, 0, 0)), dtype=float)
    M = np.linalg.inv(C) @ M
    eigval, eigvec = np.linalg.eig(M)
    con = 4 * eigvec[0]* eigvec[2] - eigvec[1]**2
    ak = eigvec[:, np.nonzero(con > 0)[0]]
    return np.concatenate((ak, T @ ak)).ravel()


def cart_to_pol(coeffs):
    """

    Convert the cartesian conic coefficients, (a, b, c, d, e, f), to the
    ellipse parameters, where F(x, y) = ax^2 + bxy + cy^2 + dx + ey + f = 0.
    The returned parameters are x0, y0, ap, bp, e, phi, where (x0, y0) is the
    ellipse centre; (ap, bp) are the semi-major and semi-minor axes,
    respectively; e is the eccentricity; and phi is the rotation of the semi-
    major axis from the x-axis.

    """

    # We use the formulas from https://mathworld.wolfram.com/Ellipse.html
    # which assumes a cartesian form ax^2 + 2bxy + cy^2 + 2dx + 2fy + g = 0.
    # Therefore, rename and scale b, d and f appropriately.
    a = coeffs[0]
    b = coeffs[1] / 2
    c = coeffs[2]
    d = coeffs[3] / 2
    f = coeffs[4] / 2
    g = coeffs[5]

    den = b**2 - a*c
    if den > 0:
        raise ValueError('coeffs do not represent an ellipse: b^2 - 4ac must'
                         ' be negative!')

    # The location of the ellipse centre.
    x0, y0 = (c*d - b*f) / den, (a*f - b*d) / den

    num = 2 * (a*f**2 + c*d**2 + g*b**2 - 2*b*d*f - a*c*g)
    fac = np.sqrt((a - c)**2 + 4*b**2)
    # The semi-major and semi-minor axis lengths (these are not sorted).
    ap = np.sqrt(num / den / (fac - a - c))
    bp = np.sqrt(num / den / (-fac - a - c))

    # Sort the semi-major and semi-minor axis lengths but keep track of
    # the original relative magnitudes of width and height.
    width_gt_height = True
    if ap < bp:
        width_gt_height = False
        ap, bp = bp, ap

    # The eccentricity.
    r = (bp/ap)**2
    if r > 1:
        r = 1/r
    e = np.sqrt(1 - r)

    # The angle of anticlockwise rotation of the major-axis from x-axis.
    if b == 0:
        phi = 0 if a < c else np.pi/2
    else:
        phi = np.arctan((2.*b) / (a - c)) / 2
        if a > c:
            phi += np.pi/2
    if not width_gt_height:
        # Ensure that phi is the angle to rotate to the semi-major axis.
        phi += np.pi/2
    phi = phi % np.pi

    return x0, y0, ap, bp, e, phi


def get_ellipse_pts(params, npts=100, tmin=0, tmax=2*np.pi):
    """
    Return npts points on the ellipse described by the params = x0, y0, ap,
    bp, e, phi for values of the parametric variable t between tmin and tmax.

    """

    x0, y0, ap, bp, e, phi = params
    # A grid of the parametric variable, t.
    t = np.linspace(tmin, tmax, npts)
    x = x0 + ap * np.cos(t) * np.cos(phi) - bp * np.sin(t) * np.sin(phi)
    y = y0 + ap * np.cos(t) * np.sin(phi) + bp * np.sin(t) * np.cos(phi)
    return x, y


if __name__ == '__main__':
    # Test the algorithm with an example elliptical arc.
    npts = 250
    tmin, tmax = np.pi/6, 4 * np.pi/3
    x0, y0 = 4, -3.5
    ap, bp = 7, 3
    phi = np.pi / 4
    # Get some points on the ellipse (no need to specify the eccentricity).
    x, y = get_ellipse_pts((x0, y0, ap, bp, None, phi), npts, tmin, tmax)
    noise = 0.1
    x += noise * np.random.normal(size=npts) 
    y += noise * np.random.normal(size=npts)

    coeffs = fit_ellipse(x, y)
    print('Exact parameters:')
    print('x0, y0, ap, bp, phi =', x0, y0, ap, bp, phi)
    print('Fitted parameters:')
    print('a, b, c, d, e, f =', coeffs)
    x0, y0, ap, bp, e, phi = cart_to_pol(coeffs)
    print('x0, y0, ap, bp, e, phi = ', x0, y0, ap, bp, e, phi)

    plt.plot(x, y, 'x')     # given points
    x, y = get_ellipse_pts((x0, y0, ap, bp, e, phi))
    plt.plot(x, y)
    fig.show()


Exact parameters:
x0, y0, ap, bp, phi = 4 -3.5 7 3 0.7853981633974483
Fitted parameters:
a, b, c, d, e, f = [ -0.51733144   0.69491003  -0.49946794   6.48425125  -6.25240455
 -16.0905626 ]
x0, y0, ap, bp, e, phi =  3.872598607926932 -3.565090583511025 6.8787781952515274 2.9817110201407613 0.9011703022103111 0.7982484342121114


In [78]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5), dpi=200)

mask = data[:, 2] == 13
x0, y0, ap, bp, e, phi = cart_to_pol(fit_ellipse(data[:, 1][mask], data[:, 0][mask]))
x, y = get_ellipse_pts((x0, y0, ap, bp, e, phi))

im = ax.imshow(fixed_img)
fig.colorbar(im, ax=ax)
ax.scatter(data[:, 1][mask], data[:, 0][mask], c='r', s=1)

ax.plot(x, y)

fig.show()

In [ ]:
def fit_concentric_rings():

    

    def polar_ellispe_1d(chi, a, e, phi):
    # chi is the azimuthal angle
    # a is the major axis length
    # e is the eccentricity
    # phi is the rotation angle...I think...
    return a * (1 - (e ** 2)) / (1 - e * np.cos(chi - phi))

In [136]:
list(range(5))

[0, 1, 2, 3, 4]

In [249]:
from sklearn.metrics.pairwise import euclidean_distances
spots = peak_local_max(fixed_img, threshold_rel=0.001)
dist = euclidean_distances(spots)

spot_indices = list(range(len(spots)))
max_dist = 50
max_neighbors = 3

dist[dist > max_dist] = np.nan
dist[dist == 0] = np.nan

data = np.empty((len(spots), 3))
data[:] = np.nan
data[:, :-1] = spots

labels=[0]
for i in spot_indices:
    connections = dist[i] < max_dist
    if sum(connections) > max_neighbors:
        connections = dist[i] <= sorted(dist[i][connections])[max_neighbors - 1]
    if np.all(np.isnan(data[connections][:, -1])):
        for index, con in enumerate(connections):
            if con:
                data[index, -1] = labels[-1]
        labels.append(labels[-1] + 1)
    else:
        base_label = np.nanmin(data[connections][:, -1])
        for index, con in enumerate(connections):
            if con:
                data[index, -1] = base_label

In [396]:
def label_nearest_spots(spots, max_dist=25, max_neighbors=np.inf):
    from sklearn.metrics.pairwise import euclidean_distances
    dist = euclidean_distances(spots)

    spot_indices = list(range(len(spots)))

    # Ignore distances greater than bounds
    # Ignore same pairs
    dist[dist > max_dist] = np.nan
    #dist[dist == 0] = np.nan
    # Create dataset
    data = np.empty((len(spots), 3))
    data[:] = np.nan
    data[:, :-1] = spots

    labels=[0]
    for i in spot_indices:
        # This seems unecessary...
        connections = dist[i] < max_dist

        # Useful for connectivity
        if sum(connections) > max_neighbors:
            connections = dist[i] <= sorted(dist[i][connections])[max_neighbors - 1]
            #break
            
        extra_labels = np.unique(data[connections][:, -1])
        extra_labels = extra_labels[~np.isnan(extra_labels)]

        # Find new label if all points are unlabeled
        if np.all(np.isnan(data[connections][:, -1])):
            current_label = labels[-1] + 1
            data[connections, -1] = current_label
            labels.append(current_label)
            #print('New label added')

        # Otherwise relabel all to lowest label value
        else:
            base_label = np.nanmin(data[connections][:, -1])
            data[connections, -1] = base_label
            current_label = base_label

        
        # Relabel previous labels if connected with others...
        if np.any(extra_labels != current_label):
            for extra in extra_labels:
                if np.all([extra != current_label, 
                        not np.isnan(extra), 
                        extra in labels]):
                    #print('I should be here...')
                    
                    data[[data[:, -1] == extra][0], -1] = current_label
                    labels.remove(extra)

    data = data.astype(np.int32)
    labels = np.array(labels).astype(np.int32)
    # labels is not perfectly sequential. Why???
    return data

In [371]:
# Redo to account for different distance values. Currently using pixel differences
from sklearn.metrics.pairwise import euclidean_distances
dist = euclidean_distances(new_spots)

spot_indices = list(range(len(new_spots)))

# Ignore distances greater than bounds
# Ignore same pairs
max_dist = 10
max_neighbors = 3
dist[dist > max_dist] = np.nan
#dist[dist == 0] = np.nan
# Create dataset
data = np.empty((len(new_spots), 3))
data[:] = np.nan
data[:, :-1] = new_spots

labels=[0]
for i in spot_indices:
    i = np.random.randint(0, np.max(spot_indices))
    # This seems unecessary...
    connections = dist[i] < max_dist

    # Useful for connectivity
    if sum(connections) > max_neighbors:
        connections = dist[i] <= sorted(dist[i][connections])[max_neighbors - 1]
        #break
        
    extra_labels = np.unique(data[connections][:, -1])
    extra_labels = extra_labels[~np.isnan(extra_labels)]
    if len(extra_labels) > 1:
       #break
       pass

    # Find new label if all points are unlabeled
    if np.all(np.isnan(data[connections][:, -1])):
        current_label = labels[-1] + 1
        data[connections, -1] = current_label
        labels.append(current_label)
        #print('New label added')

    # Otherwise relabel all to lowest label value
    else:
        base_label = np.nanmin(data[connections][:, -1])
        data[connections, -1] = base_label
        current_label = base_label

    
    # Relabel previous labels if connected with others...
    if np.any(extra_labels != current_label):
        for extra in extra_labels:
            if np.all([extra != current_label, 
                       not np.isnan(extra), 
                       extra in labels]):
                #print('I should be here...')
                
                data[[data[:, -1] == extra][0], -1] = current_label
                labels.remove(extra)
                #print(f'Relabelled and removed label {extra}.')

    #break

fig, ax = plt.subplots(1, 1, figsize=(10, 5), dpi=200)

im = ax.imshow(fixed_img)
fig.colorbar(im, ax=ax)
ax.scatter(data[:, 1], data[:, 0], c='k', s=1)
ax.scatter(data[:, 1][connections], data[:, 0][connections], c='r', s=1)
ax.scatter(data[i, 1], data[i, 0], marker='*', c='w', s=5)

fig.show()
#data = data.astype(np.int32)
#labels = np.array(labels).astype(np.int32)
# labels is not perfectly sequential. Why???
print(len(np.unique(data[:, -1])))

369
